In [24]:
from ipynb.fs.full.gbs import *

# Experiments

### Experimental Setup

```
                       ┌─────────────┐
[Detector] ◄──[ent]──► │             │──► [Detector]
[Detector] ◄──[ent]──► │             │──► [Detector]
[Detector] ◄──[ent]──► │ Haar-Random │──► [Detector]
[Detector] ◄──[ent]──► │   Unitary   │──► [Detector]
[Detector] ◄──[ent]──► │             │──► [Detector]
[Detector] ◄──[ent]──► │             │──► [Detector]
                       └─────────────┘
```

### QM Simulation

Let $k$ be the number of entanglement sources. 
Then, the initial state is 
$$2^{-k/2}(|HH\rangle+|VV\rangle)^{\otimes k} = 2^{-k/2}(|1010\rangle+|0101\rangle)^{\otimes k},$$
but reordered so the left (first two) modes of each ent come first, followed by all the right (second two) modes. 
Mathematically, this is 
$$\frac{1}{\sqrt{2^{k}}} \sum_{x \in S} | xx \rangle$$
Where 
$S 
:= \left\{ x \in \{0,1\}^{2k} : \forall i \in \{1, \dots, k\},\ x_{2i-1} \ne x_{2i} \right\} 
= \{01,10\}^{k}$.

We then apply a $2k \times 2k$ Haar-random unitary $U_\text{haar}$ to the right modes by applying the $4k \times 4k$ unitary $U = I \oplus U_\text{haar}$.

Note that there are $2k$ photons in the system, with $k$ of those passing through $U_\text{haar}$, and no mode has more than one photon in it initially.


In [8]:
state = np.array([0,2,1,0,4,0,0])
print((state > 0).astype(int))
a = [(1,'e'), (2.4,'a'), (0.5, 'z')]
print(sorted(a))
print(a)

[0 1 1 0 1 0 0]
[(0.5, 'z'), (1, 'e'), (2.4, 'a')]
[(1, 'e'), (2.4, 'a'), (0.5, 'z')]


In [26]:
from collections import defaultdict

In [ ]:
def name_this_later(num_src, haar_U=None, N=1000):
    
    m = 4*num_src  # there are 4 modes per entanglement source (rH,rV,lH,lV)
    rm = 2*num_src # two modes exit the right side of each ent (rH,rV)
    n = 2*num_src  # each ent sends one photon out each side

    if not haar_U:
        haar_U = qr_haar(rm)
    if not is_unitary(haar_U):
        raise ValueError('haar_U must be unitary.')
    if haar_U.shape != (rm, rm):
        raise ValueError(f'Unitary must act on {rm} modes.')
        
    #### print(haar_U)

    # Simulate haar_U using QM ("theory")
    # prepare initial state
    all_states = get_fock_basis_states(n, m)
    non_zero_amp_states = []
    for i in range(2**num_src):
        state = np.zeros(m, dtype=int)
        for digit in range(num_src):
            # each entanglement source either releases H light
            # in both directions or V light in both directions
            ent_index = 2*digit
            h_or_v = (i % 2)
            state[ent_index + h_or_v] = 1
            state[rm + ent_index + h_or_v] = 1
            i >> 1
        non_zero_amp_states.append(tuple(state))
        
    #### print(non_zero_amp_states)

    initial_state = np.array([(tuple(state) in non_zero_amp_states) for state in all_states], dtype=complex) / sqrt(2**num_src)
    #### print('init', initial_state)
    U = direct_sum(np.eye(rm), haar_U) # unitary acting on all modes
    hsu_mat = get_hilbert_space_unitary_matrix(U, n)
    #### print(hsu_mat)
    final_state = hsu_mat @ initial_state
    
    final_probs = conj(final_state) * final_state
    qm_distr = defaultdict(lambda: 0)
    for i,state in enumerate(all_states):
        qm_distr[tuple((state[rm:] > 0).astype(int))] += final_probs[i]

    # Spoof haar_U using stochastic mechanics ("experiment")
    ents = [ent(N, r=1) for i in range(num_src)]
    l_ents = np.concatenate(tuple([ent[0] for ent in ents]))
    r_ents = np.concatenate(tuple([ent[1] for ent in ents]))
    r_ents = haar_U @ r_ents
    dl = threshold_detector(l_ents)
    dr = threshold_detector(r_ents)
    
    stoch_distr = Counter(map(tuple, dr.T))
    stoch_normalization = N - stoch_distr[tuple(np.zeros(rm))]
    
    distrs_sorted = []
    for det_event in qm_distr:
        distrs_sorted.append((qm_distr[det_event], stoch_distr[det_event]/N, ''.join(map(str,det_event))))
    distrs_sorted = sorted(distrs_sorted)
    
    labels = []
    qm_data = []
    stoch_data = []
    for qm_pt, stoch_pt, det_event in distrs_sorted:
        labels.append(det_event)
        qm_data.append(qm_pt)
        stoch_data.append(stoch_pt)
        
    x = np.arange(len(labels))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, qm_data, width, label='QM')
    rects2 = ax.bar(x + width/2, stoch_data, width, label='Spoof')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('Probabilities')
    ax.set_title('Comparing QM and Spoof')
    ax.set_xticks(x, labels)
    ax.legend()

    ax.bar_label(rects1, padding=3)
    ax.bar_label(rects2, padding=3)

    fig.tight_layout()

    plt.show()
    
    # sort by what's heralded
#     counts = defaultdict(lambda: defaultdict(lambda: 0))
#     for i in range(N):
#         counts[tuple(dl[i])][tuple(dr[i])] += 1
    
    #options
    # 1) assume heralded state (eg [0,1,0,1]) is true Fock state (not something indistinguishable like [0,3,0,3])
    # 2) assume true state is (equal?) superposition of all states consistent with heralded state (unconserved photon number means many hsu's)
    # 3) assume true state is (equal?) superposition of all states with num_src photons consistent with heralded state
    
    #QM sim it, turn amps to probs, then graph output pr distr behind stochastic sim raw counts normalized
    
name_this_later(3, N=1000)

In [ ]:
def simulate_ethru(num_src, haar_U=None, N=1000):
    '''Simulate Ent to Haar-Random Unitary (ETHRU experiment)
    using standard quantum mechanics (Fock states + permanent).
    
    num_src = number of entanglement sources (ents)
    m := 2*num_src, the number of modes exiting one side of the ents
        (one horizontal and one vertical component per ent per side)
    haar_U : m x m haar-random unitary 
        = a unitary to apply on one side of the ents
    
    Returns a '''
    
    m = 2*num_src

    if not haar_U:
        haar_U = qr_haar(m)
    if not is_unitary(haar_U):
        raise ValueError('haar_U must be unitary.')
    if haar_U.shape != (m, m):
        raise ValueError(f'Unitary must act on {m} modes.')

In [ ]:
def spoof_ethru(num_src, haar_U=None, N=1000):
    '''Simulate Ent to Haar-Random Unitary (ETHRU experiment)
    using stochastics (Squeezed states + threshold detectors).

    num_src = number of entanglement sources (ents)
    m := 2*num_src, the number of modes exiting one side of the ents
        (one horizontal and one vertical component per ent per side)
    haar_U : m x m haar-random unitary 
        = a unitary to apply on one side of the ents
    
    Returns a tuple with two m x N arrays of boolean values 
    representing detections on the left and right side of 
    the experiment respectively.'''

    m = 2*num_src

    if not haar_U:
        haar_U = qr_haar(m)
    if not is_unitary(haar_U):
        raise ValueError('haar_U must be unitary.')
    if haar_U.shape != (m, m):
        raise ValueError(f'Unitary must act on {m} modes.')

    ents = [ent(N, r=1) for i in range(num_src)]
    l_ents = np.concatenate(tuple([ent[0] for ent in ents]))
    r_ents = np.concatenate(tuple([ent[1] for ent in ents]))
    r_ents = haar_U @ r_ents
    
    return threshold_detector(l_ents), threshold_detector(r_ents)

In [ ]:
spoof_ethru()

In [ ]:
l_res, r_res = spoof_ethru(10, N=100)
print(l_res)

def tup_to_bin(tup):
    return "".join(str(int(item)) for item in tup)

lcounts = get_all_coincidence_counts(l_res)
sortedCounts = []
bins = []
counts = []
for tup in lcounts:
    bins.append(tup_to_bin(tup))
    counts.append(lcounts[tup])
    sortedCounts.append((lcounts[tup], tup_to_bin(tup)))
plt.bar(bins, counts)
plt.show()
# print(sorted(sortedCounts, reverse=True))
# print(get_all_coincidence_counts(r_res))
# print_all_coincidence_counts(l_res)
# print_all_coincidence_counts(r_res)

In [ ]:
# print(ent(17,r=1))
print(list(lcounts.keys())[0])